In [66]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import math
import csv
import pandas as pd

In [67]:
def quartile(sample, q):
    size = len(sample)
    sample.sort()
    
    if math.isclose(q * size, round(q*size)):
        return sample[round(q*size)]
    
    return sample[round(q*size) + 1] 

In [68]:
def get_sample_generator(size):
    cases = (
        (stats.norm, (0, 1), 'Normal_distribution'), 
        (stats.cauchy, (0, 1), 'Cauchy_distribution'), 
        (stats.laplace, (0, 1/math.sqrt(2)), 'Laplace_distribution'), 
        (stats.uniform, (-math.sqrt(3), 2*math.sqrt(3)), 'Uniform_distribution'),
        (stats.poisson, (10, 0), 'Poisson_distribution')
    )
    
    for case in cases:
        sample_generator = lambda: list(map(lambda x: round(x, 4), case[0].rvs(*case[1], size)))  # rounded sample
        yield sample_generator, case[2]


In [69]:
def research_generator(n):
    research_functions = (
    (lambda x: np.mean(x), 'mean(x)'),
    (lambda x: np.median(x), 'med(x)'),
    (lambda x: (min(x)+max(x)) / 2, 'Zr'),
    (lambda x: (quartile(x, 0.25) + quartile(x, 0.75)) / 2, 'Zq'),
    (lambda x: (sum(x[n // 4 + 1: n // 4 + 3*n]) / (n // 2)), 'Ztr')
    )
    
    for f in research_functions:
        yield f

In [70]:
def var_mean_to_csv(filename, res):
    frame = pd.DataFrame.from_dict(res)
    f1 = pd.DataFrame.from_dict(np.mean(frame)).transpose()
    f1.index =['E(x)']
    f2 = pd.DataFrame.from_dict(np.var(frame)).transpose()
    f2.index =['D(x)']
    f = pd.concat([f1,f2]).round(4)
    f.to_csv(filename + '.csv')
    


In [71]:
def do_research(size, cases, repetitions=1000):
    for get_sample, distr_name in cases(size):
        res = dict()
        for i in range(repetitions):
            sample = get_sample()

            for research in research_generator(size):
                fun, f_name = research
                try:
                    res[f_name] += [fun(sample)]
                except KeyError:
                    res[f_name] = [fun(sample)]
        var_mean_to_csv(distr_name + str(size), res)

In [72]:
repetitions = 1000
sizes = [10, 100, 1000]
for size in sizes:
    do_research(size, get_sample_generator)